Este projeto ados tem como objetivo principal desenvolver um modelo preditivo robusto capaz de classificar o risco de crédito de clientes. Utilizaremos um conjunto de dados financeiro para prever o score_credito (categorizado como Ruim, Normal ou Bom) com a máxima precisão possível, focando criticamente na identificação de clientes de Alto Risco.

Objetivo Central
Prever o score_credito e, principalmente, maximizar o Recall (Revocação) da classe minoritária "Risco Ruim" (Classe 0), minimizando assim as perdas financeiras decorrentes da concessão de crédito a clientes inadimplentes.

In [87]:
# Importando bibliotécas necessárias para o projeto
import pandas as pd
import numpy as np


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report 
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score

from imblearn.over_sampling import SMOTE
from collections import Counter

In [57]:
# Lendo a tabela
tabela = pd.read_csv('clientes.csv')
tabela.head(5)

,id_cliente,mes,idade,profissao,salario_anual,num_contas,num_cartoes,juros_emprestimo,num_emprestimos,dias_atraso,...,idade_historico_credito,investimento_mensal,comportamento_pagamento,saldo_final_mes,score_credito,emprestimo_carro,emprestimo_casa,emprestimo_pessoal,emprestimo_credito,emprestimo_estudantil
0,3392,1,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,265.0,21.46538,alto_gasto_pagamento_baixos,312.494089,Good,1,1,1,1,0
1,3392,2,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,266.0,21.46538,baixo_gasto_pagamento_alto,284.629162,Good,1,1,1,1,0
2,3392,3,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,3.0,...,267.0,21.46538,baixo_gasto_pagamento_medio,331.209863,Good,1,1,1,1,0
3,3392,4,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,5.0,...,268.0,21.46538,baixo_gasto_pagamento_baixo,223.451310,Good,1,1,1,1,0
4,3392,5,23.0,cientista,19114.12,3.0,4.0,3.0,4.0,6.0,...,269.0,21.46538,alto_gasto_pagamento_medio,341.489231,Good,1,1,1,1,0


In [58]:
# Veriricando as informações da tabela para trastamento se necessário
tabela.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id_cliente                100000 non-null  int64  
 1   mes                       100000 non-null  int64  
 2   idade                     100000 non-null  float64
 3   profissao                 100000 non-null  object 
 4   salario_anual             100000 non-null  float64
 5   num_contas                100000 non-null  float64
 6   num_cartoes               100000 non-null  float64
 7   juros_emprestimo          100000 non-null  float64
 8   num_emprestimos           100000 non-null  float64
 9   dias_atraso               100000 non-null  float64
 10  num_pagamentos_atrasados  100000 non-null  float64
 11  num_verificacoes_credito  100000 non-null  float64
 12  mix_credito               100000 non-null  object 
 13  divida_total              100000 non-null  fl

In [59]:
tabela.columns

Index(['id_cliente', 'mes', 'idade', 'profissao', 'salario_anual',
       'num_contas', 'num_cartoes', 'juros_emprestimo', 'num_emprestimos',
       'dias_atraso', 'num_pagamentos_atrasados', 'num_verificacoes_credito',
       'mix_credito', 'divida_total', 'taxa_uso_credito',
       'idade_historico_credito', 'investimento_mensal',
       'comportamento_pagamento', 'saldo_final_mes', 'score_credito',
       'emprestimo_carro', 'emprestimo_casa', 'emprestimo_pessoal',
       'emprestimo_credito', 'emprestimo_estudantil'],
      dtype='object')

In [60]:
tabela['score_credito'].value_counts()

score_credito
Standard    53174
Poor        28998
Good        17828
Name: count, dtype: int64

In [61]:
tabela['mix_credito'].value_counts()

mix_credito
Normal    45848
Bom       30384
Ruim      23768
Name: count, dtype: int64

In [62]:
# Criando a tabela dicionário

# Lista de dicionários
data_dict_list = [
    {'Coluna': 'id_cliente', 'Tipo': 'int64', 'Descrição': 'Identificador único do cliente.', 'Notas': 'Deve ser removida ou usada apenas como índice.'},
    {'Coluna': 'mes', 'Tipo': 'int64', 'Descrição': 'Mês de referência.', 'Notas': 'Pode ser útil se houver sazonalidade.'},
    {'Coluna': 'idade', 'Tipo': 'float64', 'Descrição': 'Idade do cliente.', 'Notas': 'Variável numérica contínua.'},
    {'Coluna': 'profissao', 'Tipo': 'object', 'Descrição': 'Categoria ou tipo de profissão.', 'Notas': 'Variável categórica. Necessita de One-Hot Encoding.'},
    {'Coluna': 'salario_anual', 'Tipo': 'float64', 'Descrição': 'Renda anual total do cliente.', 'Notas': 'Variável numérica contínua.'},
    {'Coluna': 'num_contas', 'Tipo': 'float64', 'Descrição': 'Número total de contas financeiras ativas.', 'Notas': 'Variável numérica discreta.'},
    {'Coluna': 'num_cartoes', 'Tipo': 'float64', 'Descrição': 'Número total de cartões de crédito/débito.', 'Notas': 'Variável numérica discreta.'},
    {'Coluna': 'juros_emprestimo', 'Tipo': 'float64', 'Descrição': 'Taxa de juros média paga em empréstimos existentes.', 'Notas': 'Importante indicador de risco.'},
    {'Coluna': 'num_emprestimos', 'Tipo': 'float64', 'Descrição': 'Número de empréstimos ativos.', 'Notas': 'Variável numérica discreta.'},
    {'Coluna': 'dias_atraso', 'Tipo': 'float64', 'Descrição': 'Média de dias de atraso nos pagamentos.', 'Notas': 'Alta relevância para o risco.'},
    {'Coluna': 'num_pagamentos_atrasados', 'Tipo': 'float64', 'Descrição': 'Número total de pagamentos em atraso.', 'Notas': 'Alta relevância para o risco.'},
    {'Coluna': 'num_verificacoes_credito', 'Tipo': 'float64', 'Descrição': 'Número de consultas/verificações de crédito recentes.', 'Notas': 'Variável numérica discreta.'},
    {'Coluna': 'mix_credito', 'Tipo': 'object', 'Descrição': 'O tipo de crédito utilizado (e.g., Poor, Standard, Good).', 'Notas': 'Variável categórica **ordinal**.'},
    {'Coluna': 'divida_total', 'Tipo': 'float64', 'Descrição': 'Valor total de dívidas pendentes.', 'Notas': 'Variável numérica contínua.'},
    {'Coluna': 'taxa_uso_credito', 'Tipo': 'float64', 'Descrição': 'Porcentagem do crédito total disponível sendo usado.', 'Notas': 'Razão importante, alta relevância para o risco.'},
    {'Coluna': 'idade_historico_credito', 'Tipo': 'float64', 'Descrição': 'Idade (em anos ou meses) do histórico de crédito do cliente.', 'Notas': 'Variável numérica contínua.'},
    {'Coluna': 'investimento_mensal', 'Tipo': 'float64', 'Descrição': 'Valor que o cliente investe por mês.', 'Notas': 'Pode indicar estabilidade financeira.'},
    {'Coluna': 'comportamento_pagamento', 'Tipo': 'object', 'Descrição': 'Histórico e comportamento geral de pagamento.', 'Notas': 'Variável categórica. Necessita de codificação.'},
    {'Coluna': 'saldo_final_mes', 'Tipo': 'float64', 'Descrição': 'Saldo restante no final do mês.', 'Notas': 'Variável numérica contínua.'},
    {'Coluna': 'score_credito', 'Tipo': 'object', 'Descrição': '**VARIÁVEL-ALVO (TARGET)**. O score final de crédito (e.g., Poor, Standard, Good).', 'Notas': 'Variável categórica **ordinal**. Deve ser convertida para binária (0 ou 1) para o modelo.'},
    {'Coluna': 'emprestimo_carro', 'Tipo': 'int64', 'Descrição': 'Possui (1) ou não (0) um empréstimo de carro.', 'Notas': 'Variável binária.'},
    {'Coluna': 'emprestimo_casa', 'Tipo': 'int64', 'Descrição': 'Possui (1) ou não (0) um empréstimo imobiliário.', 'Notas': 'Variável binária.'},
    {'Coluna': 'emprestimo_pessoal', 'Tipo': 'int64', 'Descrição': 'Possui (1) ou não (0) um empréstimo pessoal.', 'Notas': 'Variável binária.'},
    {'Coluna': 'emprestimo_credito', 'Tipo': 'int64', 'Descrição': 'Possui (1) ou não (0) um empréstimo de crédito rotativo.', 'Notas': 'Variável binária.'},
    {'Coluna': 'emprestimo_estudantil', 'Tipo': 'int64', 'Descrição': 'Possui (1) ou não (0) um empréstimo estudantil.', 'Notas': 'Variável binária.'}
]
df_dicionario = pd.DataFrame(data_dict_list)
df_dicionario

,Coluna,Tipo,Descrição,Notas
0,id_cliente,int64,Identificador único do cliente.,Deve ser removida ou usada apenas como índice.
1,mes,int64,Mês de referência.,Pode ser útil se houver sazonalidade.
2,idade,float64,Idade do cliente.,Variável numérica contínua.
3,profissao,object,Categoria ou tipo de profissão.,Variável categórica. Necessita de One-Hot Enco...
4,salario_anual,float64,Renda anual total do cliente.,Variável numérica contínua.
5,num_contas,float64,Número total de contas financeiras ativas.,Variável numérica discreta.
6,num_cartoes,float64,Número total de cartões de crédito/débito.,Variável numérica discreta.
7,juros_emprestimo,float64,Taxa de juros média paga em empréstimos existe...,Importante indicador de risco.
8,num_emprestimos,float64,Número de empréstimos ativos.,Variável numérica discreta.
9,dias_atraso,float64,Média de dias de atraso nos pagamentos.,Alta relevância para o risco.


In [63]:
# Agregação: Usar APENAS a última observação por cliente

tabela = tabela.sort_values(by=['id_cliente', 'mes'], ascending=True).drop_duplicates(subset=['id_cliente'], keep='last')
tabela = tabela.drop(['id_cliente', 'mes'], axis=1)

In [64]:
# Categóricas Nominais (Profissão e Comportamento): Usar One-Hot Encoding
tabela = pd.get_dummies(tabela, columns=['profissao', 'comportamento_pagamento'], drop_first=True)

In [65]:
# Categórica Ordinal (Mix_Credito): Mapeamento explícito para manter a ordem
mix_map = {'Ruim': 1, 'Normal': 2, 'Bom': 3}
tabela['mix_credito'] = tabela['mix_credito'].map(mix_map)

In [66]:
# Codificação da Variável-Alvo
codificador = LabelEncoder()
tabela['score_credito'] = codificador.fit_transform(tabela['score_credito'])

In [67]:
tabela.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12500 entries, 14751 to 54223
Data columns (total 40 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   idade                                                12500 non-null  float64
 1   salario_anual                                        12500 non-null  float64
 2   num_contas                                           12500 non-null  float64
 3   num_cartoes                                          12500 non-null  float64
 4   juros_emprestimo                                     12500 non-null  float64
 5   num_emprestimos                                      12500 non-null  float64
 6   dias_atraso                                          12500 non-null  float64
 7   num_pagamentos_atrasados                             12500 non-null  float64
 8   num_verificacoes_credito                             12500 non-null

In [68]:
# Selecionando as variaáveis para treinar o modelo
x = tabela.drop(['score_credito'], axis=1)
y = tabela['score_credito']

x_treino, x_teste, y_treino, y_teste = train_test_split(x,y, test_size=0.2, random_state=1,stratify=y)

In [69]:
# Criabdo o modelo em dois tipos de treinamentos para comparação

modelo_arvore = RandomForestClassifier(n_estimators=100, random_state=1)
modelo_Knn = KNeighborsClassifier()

#treinando os modelos
modelo_arvore.fit(x_treino, y_treino)
modelo_Knn.fit(x_treino, y_treino)

KNeighborsClassifier()

In [70]:
# Cálculando as previsões

previsao_arvore = modelo_arvore.predict(x_teste)
previsao_Knn = modelo_Knn.predict(x_teste)

In [75]:
# Comparando as previsões com o y_teste
print(classification_report(y_teste, previsao_arvore))
print(classification_report(y_teste, previsao_Knn))

              precision    recall  f1-score   support

           0       0.57      0.46      0.51       483
           1       0.60      0.57      0.59       720
           2       0.63      0.70      0.66      1297

    accuracy                           0.61      2500
   macro avg       0.60      0.58      0.59      2500
weighted avg       0.61      0.61      0.61      2500

              precision    recall  f1-score   support

           0       0.30      0.32      0.31       483
           1       0.49      0.49      0.49       720
           2       0.55      0.54      0.55      1297

    accuracy                           0.48      2500
   macro avg       0.45      0.45      0.45      2500
weighted avg       0.49      0.48      0.49      2500



In [76]:
!pip install imbalanced-learn

In [81]:
# Como o foco é em descobrir os clientes ruins, precisamos balancear os núemros das variáveis para o treinamneto ser mais efetivo

smote = SMOTE(random_state=1, k_neighbors=5)
x_treino_smote, y_treino_smote = smote.fit_resample(x_treino, y_treino)

In [82]:
# Treinando os modelos nos DADOS COM SMOTE
modelo_arvore.fit(x_treino_smote, y_treino_smote)
modelo_Knn.fit(x_treino_smote, y_treino_smote)

KNeighborsClassifier()

In [86]:
# Avaliação Detalhada do Random Forest
print("\n--- Avaliação Detalhada do Random Forest (Após SMOTE) ---")
print(classification_report(y_teste, previsao_arvore))

# Avaliação Detalhada do K-Nearest Neighbors (KNN)
print("\n--- Avaliação Detalhada do K-Nearest Neighbors (Após SMOTE) ---")
print(classification_report(y_teste, previsao_Knn))


--- Avaliação Detalhada do Random Forest (Após SMOTE) ---
              precision    recall  f1-score   support

           0       0.51      0.56      0.54       483
           1       0.59      0.63      0.61       720
           2       0.65      0.60      0.63      1297

    accuracy                           0.60      2500
   macro avg       0.59      0.60      0.59      2500
weighted avg       0.61      0.60      0.61      2500


--- Avaliação Detalhada do K-Nearest Neighbors (Após SMOTE) ---
              precision    recall  f1-score   support

           0       0.30      0.55      0.39       483
           1       0.46      0.58      0.52       720
           2       0.55      0.30      0.39      1297

    accuracy                           0.43      2500
   macro avg       0.44      0.48      0.43      2500
weighted avg       0.48      0.43      0.43      2500



In [88]:
# Otimizando o RandomForest
scorer = make_scorer(recall_score, average='weighted')

# Definição dos Parâmetros a serem testados
param_grid = {
    'n_estimators': [100, 200, 300],         
    'max_depth': [None, 5, 10, 15],          
    'min_samples_split': [2, 5, 10],         
    'class_weight': ['balanced', None]      
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=1),
    param_grid=param_grid,
    scoring=scorer,
    cv=3,
    n_jobs=-1, 
    verbose=2 
)

In [90]:
#Treinando o modelo
grid_search.fit(x_treino_smote, y_treino_smote)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=1), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'max_depth': [None, 5, 10, 15],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring=make_scorer(recall_score, response_method='predict', average=weighted),
             verbose=2)

In [91]:
# O melhor modelo encontrado
melhor_modelo_rf = grid_search.best_estimator_

print("\n\n=============== RESULTADOS DA OTIMIZAÇÃO ===============")
print(f"Melhores Parâmetros: {grid_search.best_params_}")
print(f"Melhor Score (Recall Weighted) na Validação Cruzada: {grid_search.best_score_:.4f}")



=============== RESULTADOS DA OTIMIZAÇÃO ===============
Melhores Parâmetros: {'class_weight': 'balanced', 'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Melhor Score (Recall Weighted) na Validação Cruzada: 0.6769


In [93]:
# Avaliação do Melhor Modelo no Conjunto de Teste Original
previsao_final = melhor_modelo_rf.predict(x_teste)

print("\n--- Avaliação Detalhada do Melhor Random Forest (Final) ---")
print(classification_report(y_teste, previsao_final))


--- Avaliação Detalhada do Melhor Random Forest (Final) ---
              precision    recall  f1-score   support

           0       0.53      0.57      0.54       483
           1       0.59      0.63      0.61       720
           2       0.66      0.62      0.64      1297

    accuracy                           0.61      2500
   macro avg       0.59      0.61      0.60      2500
weighted avg       0.62      0.61      0.61      2500



In [98]:
# Identificando quais são as características mais importantes para definir o score de crédito
colunas = list(x_teste.columns)
importancia = pd.DataFrame(index = colunas, data = modelo_arvore.feature_importances_)
importancia = importancia*100
importancia

,0
idade,4.398599
salario_anual,5.087267
num_contas,5.124516
num_cartoes,5.225809
juros_emprestimo,8.973136
num_emprestimos,4.731428
dias_atraso,6.771984
num_pagamentos_atrasados,5.057985
num_verificacoes_credito,5.517027
mix_credito,3.567207


Conseguimos aumentar a capacidade do modelo de identificar clientes de alto risco (Classe 0) em 11 pontos percentuais, demonstrando que a aplicação de técnicas de balanceamento de dados (SMOTE) e otimização (GridSearchCV) é crucial para a robustez de modelos de análise de crédito.

A Precisão da Classe 0 caiu de 0.57 para 0.53. Isso significa que o modelo, ao se tornar mais agressivo na busca por clientes ruins, aumentou ligeiramente o número de Falsos Positivos (clientes bons classificados como ruins). Como o objetivo é justamente não fornecer crédito aos maus clientes, essa queda na precisão se torna aceitável.

